# imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle 
import time 
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
%run ../function_proposed_gcn.py

In [3]:
with open('../fraudTrain.pkl', 'rb') as file:
    fraudTrain = pickle.load(file)    

In [4]:
df_tr, df_tst = split_dataframe(fraudTrain, 0.001)

In [8]:
df_tr.shape,df_tst.shape

((734003, 22), (314572, 22))

In [9]:
df2, mask = concat(df_tr, df_tst)
df2['index'] = df2.index
df = df2.reset_index()

In [10]:
df.is_fraud.mean(), df_tr.is_fraud.mean(), df_tst.is_fraud.mean()

(0.005727773406766326, 0.007754736697261455, 0.000998181656345765)

In [11]:
groups = df.groupby('cc_num')

In [ ]:
edge_index = np.array([item for sublist in (compute_time_difference(group) for _, group in groups) for item in sublist])
edge_index = edge_index.astype(np.float64)

edge_index[:,2].mean()

plt.hist(edge_index[:,2])

theta = edge_index[:,2].mean()

edge_index[:,2] = (np.exp(-edge_index[:,2]/(theta)) != 1)*(np.exp(-edge_index[:,2]/(theta))).tolist()

gamma = 0.8

edge_index = torch.tensor([(int(row[0]), int(row[1])) for row in edge_index if row[2] > gamma], dtype=torch.long).t()

x = torch.tensor(df['amt'].values, dtype=torch.float).reshape(-1,1)
y = torch.tensor(df['is_fraud'].values,dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index = edge_index, y=y, train_mask = mask[0], test_mask= mask[1])


model = GCN1()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
yy = (data.y[data.test_mask]).numpy()
yyhat, yyhat_ = train_and_evaluate_model(data, model, optimizer)
yyhat_ = yyhat_.detach().numpy()
eval = evaluation(yy, yyhat, yyhat_)

eval